In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.chdir('/kaggle/input/nlp-getting-started')

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
test_data = pd.read_csv('test.csv')
train_data = pd.read_csv('train.csv')
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.drop(columns=['id', 'keyword', 'location'], inplace=True, axis = 1)
test_data.drop(columns=[ 'keyword', 'location'], inplace=True, axis = 1)
train_data.head()


In [ ]:
test_data.head()

In [ ]:
train_data['len'] = 0
j = 0
for i in train_data.text:
    train_data.len[j] = len(i)
    j +=1
train_data.head()

In [ ]:
print('maximum length of tweet is' , max(train_data.len))

In [ ]:
import seaborn as sns
sns.distplot(train_data.len)

In [ ]:
train_data['len'].describe()

In [ ]:
max_features = 10000
maxlen = 150
embedding_size = 200

In [ ]:
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_data['text'] = train_data['text'].apply(lambda x: x.lower())
    
vocabSize = 10000 #Vocabulary size
tokenizer = Tokenizer(num_words = vocabSize, split=' ')
tokenizer.fit_on_texts(train_data['text'].values) #fitting tokenizer based on the text 
X = tokenizer.texts_to_sequences(train_data['text'].values) #converting text to sequences

In [ ]:
X = pad_sequences(X, maxlen = maxlen)#making sequences of same size by padding with 0

In [ ]:
y = train_data['target']
y = np.array(y)
y.dtype

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
X[10]

In [ ]:
tokenizer.word_index

In [ ]:
num_words = len(tokenizer.word_index) + 1
print(num_words)

In [ ]:
ls ../glove6b200d/

In [ ]:
EMBEDDING_FILE = '../glove6b200d/glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, 200))

for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
	#if i >= vocabSize - 1:
		#break

In [ ]:
embedding_matrix.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input, Flatten


model = Sequential()
model.add(Input(shape=(maxlen,)))
model.add(Embedding(input_dim=num_words, output_dim=200, weights=[embedding_matrix],  trainable=True,  input_length=maxlen))
model.add(Bidirectional(LSTM(units=200)))
model.add(Dense(100, activation="softmax"))
model.add(Dropout(0.15))
model.add(Flatten())
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(X, y, batch_size=32, epochs=10)

In [ ]:
X_test = tokenizer.texts_to_sequences(test_data['text'].values)
X_test= pad_sequences(X_test, maxlen = maxlen)
y_test = (model.predict(X_test) > 0.5).astype("int32")
y_test=np.array(y_test)
y_test_id=np.array(test_data['id'].values)


In [ ]:
y_test = y_test.reshape(-1)
y_test.shape

In [ ]:
dataset = pd.DataFrame({'id': y_test_id, 'target': y_test}, columns=['id', 'target'])
dataset.head(20)

In [ ]:
dataset.to_csv('/kaggle/working/submission.csv',index = False)

In [ ]:
pwd
